In [267]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from sklearn.model_selection import train_test_split

# BertForSequenceClassification Captum

## preparing dataset

In [268]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')

In [269]:
# texts (x)
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])

# satisfaction score (y)
satisfactions_float = list(post_df['satisfaction'])
satisfactions = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)

data = []

for content, body, satisfaction in zip(post_contents, comment_bodies, satisfactions):
    data.append([content + '[SEP]' + body, satisfaction])

columns = ['contents', 'label']
df = pd.DataFrame(data, columns=columns)

# data split (train & test sets)
idx_train, idx_remain = train_test_split(df.index.values, test_size=0.20, random_state=42)
idx_val, idx_test = train_test_split(idx_remain, test_size=0.50, random_state=42)

print(idx_train.shape, idx_val.shape, idx_test.shape)

train_df = df.iloc[idx_train]
val_df = df.iloc[idx_val]
test_df = df.iloc[idx_test]

count_min_label = min(train_df['label'].value_counts())

labels = [0, 1, 2]

train_sample_df = pd.DataFrame([], columns=columns)

for label in labels:
    tmp = train_df[train_df['label'] == label]
    tmp_sampled = tmp.sample(frac=1).iloc[:count_min_label]
    train_sample_df = pd.concat([train_sample_df, tmp_sampled])

(800,) (100,) (100,)


## Loading model

In [270]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [271]:
model_path = f'../predicting-satisfaction-using-graphs/model/epoch_10.model'
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3, problem_type='multi_label_classification', output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [272]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [273]:
model.eval()
model.zero_grad()

In [274]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

## Custom forward function

In [275]:
def predict(inputs, token_type_ids=None, attention_mask=None):
    inputs = {'input_ids': inputs,
             'token_type_ids': token_type_ids,
             'attention_mask': attention_mask}
    output = model(**inputs)
        
    return output.logits

In [276]:
def classification_forward_func(inputs, token_type_ids=None, attention_mask=None):
    pred = predict(inputs, token_type_ids, attention_mask)
    # pred = np.argmax(pred.cpu().detach().numpy(), axis=1)
    # return torch.tensor(pred, device=device)
    # return pred
    return pred

## Define baselines

In [277]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference (zero padding)
sep_token_id = tokenizer.sep_token_id # A token used as a separator between post and comment and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated post-comment word sequence

In [278]:
print(ref_token_id, sep_token_id, cls_token_id)

0 102 101


In [279]:
def construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id):
    post_ids = tokenizer.encode(post, add_special_tokens=False)
    comment_ids = tokenizer.encode(comment, add_special_tokens=False)

    # construct input token ids
    input_ids = torch.tensor([[cls_token_id] + post_ids + [sep_token_id] + comment_ids + [sep_token_id]], device=device)
    seq_len = input_ids.size(1)
    
    # token_type_ids is used both embeddings
    token_type_ids = torch.tensor([[0 for i in range(seq_len)]], device=device)

    # construct reference token ids 
    ref_input_ids = torch.tensor([[cls_token_id] + [ref_token_id] * len(post_ids) + [sep_token_id] + \
        [ref_token_id] * len(comment_ids) + [sep_token_id]], device=device)

    return input_ids, ref_input_ids, token_type_ids

In [280]:
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [281]:
def construct_whole_bert_embeddings(input_ids, ref_input_ids, token_type_ids):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=token_type_ids)
    
    return input_embeddings, ref_input_embeddings

In [282]:
post, comment = post_contents[0], comment_bodies[0]

In [283]:
print(construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id))

(tensor([[  101,  2353,  2095,  2267,  3076,  1012,  2026,  6245,  2038,  2467,
         27674,  4321,  2013,  2591, 10089,  1010, 10261,  1045,  2018,  2814,
          2021,  2025,  4209,  2129,  2000,  2191,  2068,  1012,  1045,  2985,
          2048,  2086,  1999, 19568,  2015,  1998,  2196,  3764,  2000,  2151,
          1997,  2026, 10638,  1998,  2134,  1005,  1056,  2428,  3113, 10334,
          1012,  2197,  2305,  1010,  2005,  1996,  2034,  2051,  1999,  2026,
          2166,  1010,  1045,  2001,  4778,  2000,  1037,  2283,  1006,  2025,
          7714,  2568,  2017,  1025,  1045,  2001,  1037,  2112,  1997,  1037,
          2177,  2008,  2288, 13643,  4778,  1007,  1012,  1045, 10749,  1037,
          2210,  2978,  1010,  2021,  2288, 10247,  2100,  2855,  1010,  1998,
          2025,  4209,  2026,  6537,  1045,  3030,  1012,  1045,  2001,  3294,
         17358,  2306,  1996,  2034,  3178,  1012,  4661, 25795,  1998,  2019,
          8138, 14806,  1010,  1996,  2069,  2111, 

In [284]:
input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)

In [285]:
attention_mask = construct_attention_mask(input_ids)

In [286]:
indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [287]:
len(all_tokens)

285

In [288]:
len(input_ids[0])

285

## Predicting

In [289]:
pred = predict(input_ids,
               token_type_ids=token_type_ids,
               attention_mask=attention_mask)

print(pred)

tensor([[ 4.4980, -4.1010, -5.0655]], grad_fn=<AddmmBackward>)


In [290]:
pred = classification_forward_func(input_ids,
               token_type_ids=token_type_ids,
               attention_mask=attention_mask)

print(pred)

tensor([[ 4.4980, -4.1010, -5.0655]], grad_fn=<AddmmBackward>)


## Apply LayerIntegratedGradients

In [291]:
lig = LayerIntegratedGradients(classification_forward_func, model.bert.embeddings)

In [292]:
lig

In [293]:
attribution, attribution_delta = lig.attribute(inputs=input_ids, 
                                               baselines=ref_input_ids,
                                               target=2,
                                               additional_forward_args=(token_type_ids, attention_mask),
                                               return_convergence_delta=True)

In [294]:
attribution[0]

tensor([[ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         -0.0000e+00, -0.0000e+00],
        [ 1.4611e-04,  1.3571e-03,  9.8245e-04,  ...,  5.2328e-05,
          1.9038e-03,  1.2555e-03],
        [-4.8884e-04, -6.8524e-04,  5.1778e-04,  ...,  2.0780e-04,
          1.9909e-04, -1.5710e-05],
        ...,
        [-1.3424e-03,  2.6367e-04, -1.6980e-05,  ...,  3.6467e-05,
         -1.5894e-03,  1.4697e-03],
        [-1.3190e-03, -6.4167e-06,  2.8747e-04,  ..., -2.0616e-04,
          2.0345e-04,  4.1385e-05],
        [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         -0.0000e+00, -0.0000e+00]], dtype=torch.float64)

In [295]:
attribution_delta

tensor([-0.1639], dtype=torch.float64)